In [1]:
import pandas as pd
df = pd.read_csv("../Data/FIBAData.csv", index_col = 0, encoding='latin-1')
df = df.dropna(subset=['Player'])
df.head()

,Region,Date,Away Team,Home Team,Result,Team,Period,Clock,Player,Play Details,Away Score,Home Score
ID,,,,,,,,,,,,
1,2019 World Cup,9/15/2019,Spain,Argentina,Miss 2 Pts,Spain,1,9:39,Ricky Rubio,9 Ricky Rubio > Hand Off > From Stationary > R...,0,0
4,2019 World Cup,9/15/2019,Spain,Argentina,Miss 3 Pts,Argentina,1,9:30,Luis Scola,7 Facundo Campazzo > P&R Ball Handler > High P...,0,0
7,2019 World Cup,9/15/2019,Spain,Argentina,Miss 2 Pts,Spain,1,9:15,Rudy Fernandez,5 Rudy Fernandez > Cut > Basket > Miss 2 Pts,0,0
9,2019 World Cup,9/15/2019,Spain,Argentina,Run Offense,Spain,1,9:15,Juancho Hernangomez,41 Juancho Hernangomez > Offensive Rebound > S...,0,0
11,2019 World Cup,9/15/2019,Spain,Argentina,Make 2 Pts,Spain,1,9:01,Pierre Oriola,18 Pierre Oriola > Spot-Up > No Dribble Jumper...,2,0


In [2]:
play_types = ['P&R Ball Handler', 'Spot-Up', 'Transition', 'ISO', 'Post-Up', 'P&R Roll Man', 'Cut', 'Offensive Rebound', 'Off Screen', 'Hand Off']

def play_counts(player):
    """
    Return player's play type counts for game with given gameID.
    Synergy actions: PnR Handler, Spot Up, Transition, Isolation, Post Up, PnR Man, Cut, ORB, Off Screen, Hand Off
    player: String, player's full name as appears in data
    gameID: int, game ID from df
    return: dict, {play_type: count}
    """
    play_distribution = {play_type: 0 for play_type in play_types}
    temp = df[df['Play Details'].str.contains(" " + player + " ")]['Play Details']
    for play in temp.tolist():
        play = play.split(' > ')
        if player in play[0] or 'Ball Delivered' in play: # If assisted by different player, count under resulting play type
            play_type = play[1 if player in play[0] else play.index('Ball Delivered') + 2]
            if play_type in play_types:
                play_distribution[play_type] += 1
    return play_distribution

play_counts("Marc Gasol")

{'P&R Ball Handler': 2,
 'Spot-Up': 50,
 'Transition': 33,
 'ISO': 14,
 'Post-Up': 166,
 'P&R Roll Man': 115,
 'Cut': 52,
 'Offensive Rebound': 30,
 'Off Screen': 5,
 'Hand Off': 2}

In [3]:
from numpy import nan
players = df['Player'].unique().tolist()
len(players)

2873

In [4]:
full_counts = {p: play_counts(p) for p in players}
res_df = pd.DataFrame(full_counts).transpose()
res_df

,P&R Ball Handler,Spot-Up,Transition,ISO,Post-Up,P&R Roll Man,Cut,Offensive Rebound,Off Screen,Hand Off
Ricky Rubio,284,121,64,20,0,0,9,25,7,16
Luis Scola,0,122,105,19,304,167,125,97,20,1
Rudy Fernandez,109,103,54,33,0,1,20,37,36,12
Juancho Hernangomez,2,53,25,5,5,8,26,35,13,1
Pierre Oriola,0,21,10,0,3,8,18,38,1,1
...,...,...,...,...,...,...,...,...,...,...
Chin Yong Wong,0,2,1,0,0,0,0,0,0,0
Wee Seng Wong,1,5,0,0,0,0,0,0,0,0
Min Joe Foong,0,0,0,0,0,0,0,0,0,0
Lok Seng Mak,0,1,0,0,0,0,1,0,0,0


In [5]:
res_df['Total'] = res_df.sum(axis=1)

In [6]:
res_df.to_csv('FIBA_Play_Counts.csv')